#### Домашнее задание
1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других

In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb, lightgbm as lgbm, catboost as catb
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion

Загрузим данные

In [2]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


Разобьем данные на обучающую и тестовую выборки

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), df['cardio'], random_state=0)

Преобразуем данные. К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [4]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [5]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Объединим все трансформеры с помощью FeatureUnion

In [6]:
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Создадим модели классификации

In [7]:
# Модель логистической регрессии

classifier_lr = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

#запустим кросс-валидацию
cv_scores_lr = cross_val_score(classifier_lr, X_train, y_train, cv=16, scoring='roc_auc')
cv_score_lr = np.mean(cv_scores_lr)
cv_score_std_lr = np.std(cv_scores_lr)
print('CV score is {}+-{}'.format(cv_score_lr, cv_score_std_lr))

#обучим пайплайн на всем тренировочном датасете
classifier_lr.fit(X_train, y_train)
y_score_lr = classifier_lr.predict_proba(X_test)[:, 1]

CV score is 0.7867401104915408+-0.00852135511666111


In [9]:
# Модель К-ближайших соседей

classifier_knn = Pipeline([
    ('features',feats),
    ('classifier', KNeighborsClassifier()),
])

#запустим кросс-валидацию
cv_scores_knn = cross_val_score(classifier_knn, X_train, y_train, cv=16, scoring='roc_auc')
cv_score_knn = np.mean(cv_scores_knn)
cv_score_std_knn = np.std(cv_scores_knn)
print('CV score is {}+-{}'.format(cv_score_knn, cv_score_std_knn))

#обучим пайплайн на всем тренировочном датасете
classifier_knn.fit(X_train, y_train)
y_score_knn = classifier_knn.predict_proba(X_test)[:, 1]

CV score is 0.6919218138274713+-0.007083802929940452


In [12]:
# Модель рещающего дерева

classifier_tree = Pipeline([
    ('features',feats),
    ('classifier', DecisionTreeClassifier(random_state = 42, max_depth=4)),
])

#запустим кросс-валидацию
cv_scores_tree = cross_val_score(classifier_tree, X_train, y_train, cv=16, scoring='roc_auc')
cv_score_tree = np.mean(cv_scores_tree)
cv_score_std_tree = np.std(cv_scores_tree)
print('CV score is {}+-{}'.format(cv_score_tree, cv_score_std_tree))

#обучим пайплайн на всем тренировочном датасете
classifier_tree.fit(X_train, y_train)
y_score_tree = classifier_tree.predict_proba(X_test)[:, 1]

CV score is 0.7872114912561746+-0.005833774770971484


In [13]:
# Модель XGBoost

classifier_xgb = Pipeline([
    ('features',feats),
    ('classifier', xgb.XGBClassifier(random_state = 42)),
])

#запустим кросс-валидацию
cv_scores_xgb = cross_val_score(classifier_xgb, X_train, y_train, cv=16, scoring='roc_auc')
cv_score_xgb = np.mean(cv_scores_xgb)
cv_score_std_xgb = np.std(cv_scores_xgb)
print('CV score is {}+-{}'.format(cv_score_xgb, cv_score_std_xgb))

#обучим пайплайн на всем тренировочном датасете
classifier_xgb.fit(X_train, y_train)
y_score_xgb = classifier_xgb.predict_proba(X_test)[:, 1]

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:06:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:06:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:07:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:07:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
CV score is 0.7974879407860951+-0.006447877492510205
[15:07:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [14]:
# Модель LGBMClassifier

classifier_lgbm = Pipeline([
    ('features',feats),
    ('classifier', lgbm.LGBMClassifier(random_state = 42)),
])

#запустим кросс-валидацию
cv_scores_lgbm = cross_val_score(classifier_lgbm, X_train, y_train, cv=16, scoring='roc_auc')
cv_score_lgbm = np.mean(cv_scores_lgbm)
cv_score_std_lgbm = np.std(cv_scores_lgbm)
print('CV score is {}+-{}'.format(cv_score_lgbm, cv_score_std_lgbm))

#обучим пайплайн на всем тренировочном датасете
classifier_lgbm.fit(X_train, y_train)
y_score_lgbm = classifier_lgbm.predict_proba(X_test)[:, 1]

CV score is 0.8024230111465757+-0.006885787188310477


In [15]:
# Модель CatBoost

classifier_catb = Pipeline([
    ('features',feats),
    ('classifier', catb.CatBoostClassifier(silent=True, random_state = 42)),
])

#запустим кросс-валидацию
cv_scores_catb = cross_val_score(classifier_catb, X_train, y_train, cv=16, scoring='roc_auc')
cv_score_catb = np.mean(cv_scores_catb)
cv_score_std_catb = np.std(cv_scores_catb)
print('CV score is {}+-{}'.format(cv_score_catb, cv_score_std_catb))

#обучим пайплайн на всем тренировочном датасете
classifier_catb.fit(X_train, y_train)
y_score_catb = classifier_catb.predict_proba(X_test)[:, 1]

CV score is 0.8020711321539079+-0.006981171803519522


In [16]:
# Функция подсчета метрик

def metrics_calculation(y_test, y_score):

    roc_auc = roc_auc_score(y_test, y_score)
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    return thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc

In [17]:
# Подсчитываем метрики для разных моделей

all_y_scores = [y_score_lr, y_score_knn, y_score_tree, y_score_xgb, y_score_lgbm, y_score_catb]
metrics = []
for y_score in all_y_scores:
    best_threshold, fscore, precision, recall, roc_auc = metrics_calculation(y_test, y_score)
    metrics.append([fscore, precision, recall, roc_auc])

In [18]:
# Создаем сравнительную таблицу метрик

metrics_comparison_df = pd.DataFrame(np.array(metrics),
                   columns=['fscore', 'precision', 'recall', 'roc_auc'],
                   index=['y_score_lr', 'y_score_knn', 'y_score_tree', 'y_score_xgb', 'y_score_lgbm', 'y_score_catb'])
metrics_comparison_df

,fscore,precision,recall,roc_auc
y_score_lr,0.730323,0.647431,0.837558,0.784035
y_score_knn,0.685863,0.588468,0.821889,0.694088
y_score_tree,0.731045,0.646918,0.840323,0.788519
y_score_xgb,0.737890,0.665248,0.828341,0.797228
y_score_lgbm,0.739561,0.660803,0.839631,0.801689
y_score_catb,0.739538,0.691275,0.795046,0.801080


#### Вывод:

Метрика roc_auc выше всего у моделей LGBMClassifier и CATBoost